In [ ]:
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install --quiet accelerate==0.21.0
!pip install --quiet peft==0.4.0
!pip install --quiet bitsandbytes==0.40.2
!pip install --quiet transformers==4.31.0
!pip install --quiet trl==0.4.7
!pip install --quiet wandb
!pip install --quiet evaluate
!pip install --quiet xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 252.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import time

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
device_map = {"": 0}

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb = imdb.shuffle(seed=42)
dataset = imdb['train'].train_test_split(test_size=0.01, shuffle=False)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset["test"][:1]

{'text': ['There\'s no shortage of bad dialogue in David and Bathsheba \x96 "I was quite a hand with a slingshot," "The King of all Israel out there in the darkness exposing himself to the enemy" (full marks to Dennis Hooey for delivering that one with a straight face), "Go and sit with the concubines." And somehow I doubt a bored David ever told the prophet Nathan "Whatever you say." He even tries the old "My kingdom doesn\'t understand me" routine on desperate housewife Bathsheba at one point. So it\'s probably a tribute to Henry King\'s direction that the film isn\'t at all bad despite the pitfalls much of the first third provide. Maybe it\'s the censor-baiting nature of the plot \x96 a married man kills a femme fatale\'s husband and gets away with it! \x96 but King brings out the growing moral and theological complexities in Phillip Dunne\'s script rather than upping the sin and sandals hokum. This is the conflicted David on the downhill slope, abandoned by a vengeful God he no lon

util for getting predictions

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

def format_dolly(sample):
    sys_prompt = """<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>"""
    instruction = f"### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1."
    context = f"### Context\n{sample['text']}"
    response = f"### Answer\n"
    prompt = "\n\n".join([i for i in [sys_prompt, instruction, context, response] if i is not None])
    return prompt

from random import randrange
print(template_dataset(dataset["train"][randrange(len(dataset["train"]))]))

{'text': '<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nAlong with "Brothers & Sisters", "Six Degrees" was one of my favorite new dramas of fall 2006. <br /><br />Great cast all around, but really enjoyed the work of Campbell Scott (the come-back photog) and Hope Davis (recent widow of journalist killed in Iraq).<br /><br />Aside from the acting, the writing was fresh and the acting superb. The show was 

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.01,
    device_map=device_map)

In [ ]:
prompt = format_dolly({"text": dataset["test"][0]['text'], "label": ""})
result = pipe(
    f"{prompt}",
    do_sample=True,
    top_k=1,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=1024,
    temperature=0.1
)
result

[{'generated_text': '<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\n    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.<</SYS>>\n\n### Instruction\nRate the sentiment of the following text, using a 0 or 1. 1 is a good sentiment and 0 is a bad one. Do not respond with anything other than a 0 or 1.\n\n### Context\nThere\'s no shortage of bad dialogue in David and Bathsheba \x96 "I was quite a hand with a slingshot," "The King of all Israel out there in the darkness exposing himself to the enemy" (full marks to Dennis Hooey for delivering that one with a straight face), "Go and sit with the concubines." And somehow I doubt a bored David ever told th

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

iterable_dataset = dataset['test'].select(range(50)).to_iterable_dataset()
refs = []
preds = []
for sample in iterable_dataset:
    prompt = format_dolly({"text": sample['text'], "label": ""})
    result = pipe(
        f"{prompt}",
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
        temperature=0.1
    )

    answer = result[0]['generated_text']
    answer = answer.split("### Answer\n")[1]
    answer = answer.replace(".", "")

    if "###" in answer:
        answer, explanation = answer.split("###")
        print(f"Explanation {explanation}")

    if len(answer) > 0:
        answer = answer[0].strip()

    if not answer.isdigit():
        print(f"HAD NO DIGIT\n\n{result}")
        answer = -10

    preds.append(answer)
    refs.append(sample['label'])


accuracy.add_batch(references=refs, predictions=preds)
print(f"{accuracy.compute()}")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Input length of input_ids is 1238, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 1386, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 1363, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


{'accuracy': 0.46}
